In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, MaxPool2D
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [6]:
def import_maps(path):
    images = []
    for index, name in enumerate(os.listdir(path)):
        folder = os.path.join(path, name)
        for file_class in os.listdir(folder):
            im_folder = os.path.join(folder, file_class)

            for im in os.listdir(im_folder):
                img = cv2.imread(os.path.join(im_folder, im))
                img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                img = cv2.resize(img, (196, 196))
                if img is not None:
                    images.append((np.array(img), index)) 
    return images

In [7]:
image_train_set = import_maps(r'D:\ILBS_14_8_21_Sanjeev\ILBS_Clustered_Volumes\train')
image_test_set = import_maps(r'D:\ILBS_14_8_21_Sanjeev\ILBS_Clustered_Volumes\test')

In [8]:
len(image_train_set)

12900

In [9]:
len(image_test_set)

1950

In [10]:
train_images_all = [i[0] for i in image_train_set]
train_images_array = np.array(train_images_all)
#train_images_array=np.expand_dims(train_images_array,axis=3)
train_image_label = [i[1] for i in image_train_set]
train_image_label = np.array(train_image_label)

In [11]:
test_images_all = [i[0] for i in image_test_set]
test_images_array = np.array(test_images_all)
#test_images_array=np.expand_dims(test_images_array,axis=3)
test_image_label = [i[1] for i in image_test_set]
test_image_label = np.array(test_image_label)

In [12]:
scalar = StandardScaler(copy=False)
train_images = scalar.fit_transform(train_images_array.reshape(12900, 196*196))
train_images_array = train_images.reshape(12900, 196, 196, 1)

In [13]:
test_image = scalar.fit_transform(test_images_array.reshape(1950,196*196))
test_image_array = test_image.reshape(1950,196,196,1)

In [14]:
train_x, val_x, train_y, val_y = train_test_split(train_images_array,train_image_label,test_size=0.2)

In [15]:
enc = OneHotEncoder(handle_unknown='ignore')
train_label_enc = enc.fit_transform(train_y.reshape(-1, 1)).toarray()
val_label_enc = enc.fit_transform(val_y.reshape(-1, 1)).toarray()
test_label_enc = enc.fit_transform(test_image_label.reshape(-1, 1)).toarray()

In [16]:
model = Sequential()
model.add(Conv2D(input_shape=(196,196,1),filters=32,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='vgg16'))
model.add(Flatten(name='flatten'))
model.add(Dense(512, activation='relu', name='fc1'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu', name='fc2'))
model.add(Dropout(0.25))
model.add(Dense(6, activation='softmax', name='output'))

model.compile(loss=keras.losses.categorical_crossentropy,
               optimizer=keras.optimizers.SGD(0.001),
               metrics=['accuracy'])

model.summary()

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                            patience=15, min_lr=0.00001)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 196, 196, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 196, 196, 64)      18496     
_________________________________________________________________
batch_normalization (BatchNo (None, 196, 196, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 98, 98, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 98, 98, 128)       147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 98, 98, 128)       5

In [17]:
history = model.fit(train_x,train_label_enc,validation_data=(val_x,val_label_enc), batch_size=64, epochs=60, verbose=1,shuffle=True, callbacks = [reduce_lr])

Train on 10320 samples, validate on 2580 samples
Epoch 1/60
10320/10320 [==============================] - 77s 7ms/sample - loss: 1.3466 - accuracy: 0.5097 - val_loss: 2.0336 - val_accuracy: 0.0372
Epoch 2/60
10320/10320 [==============================] - 60s 6ms/sample - loss: 0.4811 - accuracy: 0.8396 - val_loss: 2.0755 - val_accuracy: 0.0504
Epoch 3/60
10320/10320 [==============================] - 60s 6ms/sample - loss: 0.1924 - accuracy: 0.9413 - val_loss: 1.0580 - val_accuracy: 0.7674
Epoch 4/60
10320/10320 [==============================] - 60s 6ms/sample - loss: 0.1380 - accuracy: 0.9555 - val_loss: 0.0957 - val_accuracy: 0.9760
Epoch 5/60
10320/10320 [==============================] - 60s 6ms/sample - loss: 0.1005 - accuracy: 0.9657 - val_loss: 0.0519 - val_accuracy: 0.9744
Epoch 6/60
10320/10320 [==============================] - 60s 6ms/sample - loss: 0.0818 - accuracy: 0.9705 - val_loss: 0.0494 - val_accuracy: 0.9736
Epoch 7/60
10320/10320 [==============================] -

10320/10320 [==============================] - 61s 6ms/sample - loss: 0.0382 - accuracy: 0.9770 - val_loss: 0.0435 - val_accuracy: 0.9643
Epoch 56/60
10320/10320 [==============================] - 60s 6ms/sample - loss: 0.0389 - accuracy: 0.9779 - val_loss: 0.0436 - val_accuracy: 0.9647
Epoch 57/60
10320/10320 [==============================] - 60s 6ms/sample - loss: 0.0372 - accuracy: 0.9774 - val_loss: 0.0436 - val_accuracy: 0.9643
Epoch 58/60
10320/10320 [==============================] - 60s 6ms/sample - loss: 0.0364 - accuracy: 0.9793 - val_loss: 0.0437 - val_accuracy: 0.9643
Epoch 59/60
10320/10320 [==============================] - 60s 6ms/sample - loss: 0.0369 - accuracy: 0.9777 - val_loss: 0.0439 - val_accuracy: 0.9643
Epoch 60/60
10320/10320 [==============================] - 60s 6ms/sample - loss: 0.0384 - accuracy: 0.9772 - val_loss: 0.0439 - val_accuracy: 0.9647


In [22]:
score = model.evaluate(test_image_array,test_label_enc)

1950/1950 [==============================] - 5s 3ms/sample - loss: 2.9606 - accuracy: 0.3185


In [31]:
test_label_enc[551:560]

array([[0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]])

In [40]:
test_image_label[100:760]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [56]:
count=0
for i in test_image_label:
    if i==5:
        count +=1
    else:
        pass
print(count)

300


In [58]:
#def import_maps(path):
path = r'D:\ILBS_14_8_21_Sanjeev\ILBS_Clustered_Volumes\test'
images = []
for index, name in enumerate(os.listdir(path)):
    print(index)
    folder = os.path.join(path, name)
    print(folder)
#     for file_class in os.listdir(folder):
#         im_folder = os.path.join(folder, file_class)

#         for im in os.listdir(im_folder):
#             img = cv2.imread(os.path.join(im_folder, im))
#             img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#             img = cv2.resize(img, (196, 196))
#             if img is not None:
#                 images.append((np.array(img), index)) 
#     return images

0
D:\ILBS_14_8_21_Sanjeev\ILBS_Clustered_Volumes\test\00_5
1
D:\ILBS_14_8_21_Sanjeev\ILBS_Clustered_Volumes\test\06_10
2
D:\ILBS_14_8_21_Sanjeev\ILBS_Clustered_Volumes\test\11_15
3
D:\ILBS_14_8_21_Sanjeev\ILBS_Clustered_Volumes\test\16_20
4
D:\ILBS_14_8_21_Sanjeev\ILBS_Clustered_Volumes\test\21_25
5
D:\ILBS_14_8_21_Sanjeev\ILBS_Clustered_Volumes\test\26_30
